In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

def set_css_in_cell_output():
    display(HTML('''
        <style>
            .jupyter-widgets {color: #d5d5d5 !important;}
            .widget-label {color: #d5d5d5 !important;}
        </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)
from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#0D0D0D"

In [2]:
import pickle
import time
import tiktoken
import datetime
import json
import requests
import traceback
import re
import numpy as np
from tqdm.notebook import tqdm
from jsonschema import validate
from openai import OpenAI, RateLimitError, APITimeoutError, InternalServerError, Timeout
from tenacity import retry, stop_after_attempt, wait_incrementing, retry_if_exception_type, after_log, before_sleep_log
import logging
import mysql.connector
from mysql.connector import Error
from IPython.display import clear_output

In [ ]:
OPENROUTER_API_KEY = ''        # private openrouter api key

In [4]:
request_limit_per_minute = 500
token_limit_per_minute = 2e6

request_timeout_seconds = 120   # maximum wait time for openAI to respond before triggering request timeout 
request_max_retries = 1         # number to times to automatically retry failed requests
tpm_wait_polling_seconds = 10    # if our internal TPM estimate thinks TPM limit is exceeded, how often to check if limit cleared

# global logger for static classes
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

In [6]:
class ChatGPT:
    def __init__(self, model_provider_order,
                 halt_on_error=True,
                 is_verbose=True,
                 timeout=request_timeout_seconds,
                 max_retries=request_max_retries,
                 request_limit_per_minute=request_limit_per_minute,
                 token_limit_per_minute=token_limit_per_minute,
                 tpm_wait_polling_seconds=tpm_wait_polling_seconds,
                 logger=logger,
                 api_key=OPENROUTER_API_KEY,
                 limit_manager_db_password=LIMIT_MANAGER_DB_PASSWORD):
        self.model, self.provider_order, self.model_canonical_name = model_provider_order
        self.halt_on_error = halt_on_error
        self.is_verbose = is_verbose
        self.tpm_wait_polling_seconds = tpm_wait_polling_seconds
        self.request_limit_per_minute = request_limit_per_minute
        self.request_delay_seconds = 60.0 / request_limit_per_minute
        self.token_limit_per_minute = token_limit_per_minute
        self.response_history = []
        self.message_history = {}
        self.logger = logger
        self.limit_manager_db_password = limit_manager_db_password
        likert_options = [
            "Very Inaccurate",
            "Moderately Inaccurate",
            "Neither Accurate nor Inaccurate",
            "Moderately Accurate",
            "Very Accurate",
        ]
        # Sort by length to match longer options (e.g., "Strongly Agree") before shorter ones (e.g., "Agree")
        self.likert_options = sorted(likert_options, key=len, reverse=True)
        self.default_seed = 1 if 'x-ai' in self.model else 0
        
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1",
                             api_key = api_key,
                             timeout=timeout,
                             max_retries=max_retries)

        
    def extract_likert_response(self, content):
        content_lower = content.lower()
        for option in self.likert_options:
            pattern = r'\b' + re.escape(option.lower()) + r'\b'
            match = re.search(pattern, content_lower)
            if match:
                return json.dumps({"response": option})
        raise Exception("No Likert match found in: ", content)    
    
    
    def get_running_cost_num_prompt_completion_tokens(self):
        """
        This function computes the total cost (estimated) of all
        messages sent by the instance of ChatGPT called from
        Returns: total_running_cost, total_num_prompt_tokens, total_num_response_tokens
        """
        n_prompt_tokens = np.sum([x.usage.prompt_tokens for x in self.response_history])
        n_completion_tokens = np.sum([x.usage.completion_tokens for x in self.response_history])
        total_cost = sum(r.usage.cost for r in self.response_history)
        return (total_cost,
                n_prompt_tokens,
                n_completion_tokens)

    def get_key_usage_credits(self):
        # get what OpenRouter says the api key has used in total
        # returns usage, total credits available
        resp = requests.get(
            "https://openrouter.ai/api/v1/credits",
            headers={"Authorization": f"Bearer {self.client.api_key}"}
        )
        resp.raise_for_status()
        info = resp.json()["data"]
        return info["total_usage"], info["total_credits"]

    # retry failing requests starting with 10 second wait,
    # increasing wait time by 10 seconds each retry, up to a max window of 120s (or 5 times)
    # the goal is to try to avoid hitting backoff,
    # we treat this as a last resort because of its runtime cost
    @retry(wait=wait_incrementing(start=10, increment=10, max=120),
           stop=stop_after_attempt(5),
           retry=retry_if_exception_type((RateLimitError, APITimeoutError, InternalServerError, Timeout)),
           before_sleep=before_sleep_log(logger, logging.INFO),
           after=after_log(logger, logging.INFO))
    def completion_with_backoff(self, client, **kwargs):
        return client.chat.completions.create(**kwargs)


    def check_internal_TPM_tracker(self, n_message_tokens):
        """
        Checks internal TPM count to see if a message with length = n_message_tokens
        can be sent. If not, it waits (sleeps - blocking) until the message delivery
        meets into TPM limit
        """
        now = datetime.datetime.now()
        one_minute_ago = now + datetime.timedelta(seconds=-60)
        self.token_count_history = [x for x in self.token_count_history if x[1] > one_minute_ago]
        n_tokens_past_minute = np.sum([x[0] for x in self.token_count_history]) + n_message_tokens
        # fixed delay waiting if TPM exceeded over past minute
        # this is cpu polling, so it doesnt cost money or much compute
        while n_tokens_past_minute > self.token_limit_per_minute:
            if self.is_verbose: self.logger.info(f'Internal TPM limit exceeded, waiting for {self.tpm_wait_polling_seconds} seconds...')
            time.sleep(self.tpm_wait_polling_seconds)
            now = datetime.datetime.now()
            one_minute_ago = now + datetime.timedelta(seconds=-60)
            self.token_count_history = [x for x in self.token_count_history if x[1] > one_minute_ago]
            n_tokens_past_minute = np.sum([x[0] for x in self.token_count_history]) + n_message_tokens
        now = datetime.datetime.now()
        self.token_count_history.append((n_message_tokens, now))


    def send_message(self, system_role, message, validate_response=True):
        """
        This is the primary function used to send messages to GPT and get responses
        Steps are:
          - check that json schema meets our basic requirements
          - handle RPM and TPM limits as best as we can
            (when openai rejects requests for exceeding limits its much slower)
          - build and send the message using openai ChatCompletion api
          - perform basic validation on GPT's response
        This function either returns a ChatCompletion response object or None (if failure occurred)
        Errors are propogated using raised Exceptions
        """
        # sleep based on RPM limit (lazy logic, avoids keeping running count of actual requests per minute)
        time.sleep(self.request_delay_seconds)

        # check TPM limit (not lazy, uses running count of tokens per minute)
        try:
            encoding = tiktoken.encoding_for_model(self.model)
        except:
            encoding = tiktoken.encoding_for_model('gpt-4')
        n_message_tokens = len(encoding.encode(system_role)) + len(encoding.encode(message))
        self.logger.info(f'processing message with {n_message_tokens} tokens...')
        if n_message_tokens > self.token_limit_per_minute:
            return self.bad_response_output(f'Unable to send message as it exceeds TPM. Number of tokens in message = {n_message_tokens}')
        
        # build and send message over openai-api
        message_id = len(self.message_history.keys())
        self.message_history[message_id] = [] if 'x-ai' in self.model else [{"role": "system", "content": system_role}]
        self.message_history[message_id].append({"role": "user", "content": message})
        try:
            response = self.completion_with_backoff(self.client,
                                                    model=self.model,
                                                    messages=self.message_history[message_id],
                                                    temperature=0,
                                                    stream=False,
                                                    extra_body={"usage": {"include": True},
                                                                "reasoning": {# One of the following (not both):
                                                                              "effort": "medium", # Can be "high", "medium", or "low" (OpenAI-style)
                                                                              # Optional: Default is false. All models support this.
                                                                              "exclude": False # Set to true to exclude reasoning tokens from response
                                                                              },
                                                                "provider": {"order": self.provider_order, 
                                                                             "sort": "price",
                                                                             "data_collection": "deny",
                                                                             "allow_fallbacks": False}},
                                                    seed=self.default_seed, logprobs=False)
            
            self.response_history.append(response)
            # reasoning models dont return a content field
            if response.choices[0].message.content is None:
                self.bad_response_output(f'None in message content')
                return None
            elif response.choices[0].message.content == '':
                if hasattr(response.choices[0].message, 'reasoning'):
                    if response.choices[0].message.reasoning != '':
                        response.choices[0].message.content = response.choices[0].message.reasoning
            else:
                pass
            try:
                likert_response = self.extract_likert_response(response.choices[0].message.content.strip().lower())
                response.choices[0].message.content = likert_response
            except:
                self.bad_response_output(f'GPT response didnt match likert options')
                return None
        except Exception as e:
            if self.halt_on_error:
                raise
            else:
                if self.is_verbose:
                    str_e = str(e)
                    self.logger.info(f'An exception occurred: {str_e}')
                    self.logger.info(traceback.format_exc())
                return None

        return (response, message_id)
        

    def bad_response_output(self, error):
        # general function for informing the user when an error occurs
        if self.halt_on_error:
            raise Exception(error)
        else:
            if self.is_verbose:
                self.logger.info(f'Error - {error}')
        return None

In [7]:
llama_4_maverick = ('meta-llama/llama-4-maverick', ['Fireworks', 'Together', 'Kluster'], 'llama_maverick')
gemini_flash = ('google/gemini-2.5-flash-preview-05-20:thinking', ['Google', 'Google AI Studio'], 'gemini_flash')
qwen_235B = ('qwen/qwen3-235b-a22b', ['DeepInfra', 'Kluster', 'Parasail', 'Together', 'Nebius'], 'qwen3_235B')  
gpt_41 = ('openai/gpt-4.1', ['OpenAI'], 'gpt_41')
gpt_41_mini = ('openai/gpt-4.1-mini', ['OpenAI'], 'gpt_41_mini')
claude_sonnet = ('anthropic/claude-3.7-sonnet', ['Anthropic', 'Amazon Bedrock', 'Google', 'Google AI Studio'], 'claude_sonnet')
grok_3 = ('x-ai/grok-3-beta', ['xAI'], 'grok3_beta')

# Example: We expect a simple string response from GPT

In [ ]:
MODEL_TO_EVALUATE = gpt_41_mini

In [8]:
def example_messaging_wrapper(chat, system_role, message):
    # with halt_on_error set to True in ChatGPT class, 
    # we use exception propogation to handle errors and edge-cases
    response, message_history_id = None, -1
    try:
        response, message_history_id = chat.send_message(system_role=system_role,
                                                         message=message, 
                                                         validate_response=True)
        assert response is not None
        response_json = json.loads(response.choices[0].message.content)
        response_str = response_json["response"]
    except Exception as e:
        response_json = {}
        response_str = ''
        chat.logger.info(f'Messaging wrapper failure - {str(e)}')
        print(traceback.format_exc())

    cost, n_prompt_tokens, n_completion_tokens = chat.get_running_cost_num_prompt_completion_tokens()
    last_messages_sent_to_gpt = '' if (message_history_id not in chat.message_history) else chat.message_history[message_history_id]
    print(f'Messages to GPT:\n{last_messages_sent_to_gpt}')
    print(f'Response from GPT:\n{response_str}')
    print(f'Cost: ${cost:.5f}')
    
    return response, message_history_id

In [9]:
# specify system role and user message
system_role = 'you are a helpful assistant.'
likert_options = [
    "Strongly Disagree",
    "Strongly Agree",
    "Disagree",
    "Neutral",
    "Agree",
]
likert_options_str = '\n'.join(likert_options)

message = """
Simulate your response to the prompt below without being provided the question and thoughts. This is for testing purposes.

---

Your task is to respond to the following IPIP-NEO-120 question based on the participant's daily diaries of the most significant event that occurred during the day, provided below. Respond as though you are the individual who generated these thoughts, reflecting their personality traits.
Base your answer on inferred personality traits. Think carefully about what the thoughts imply about tendencies and behaviors.

For each question, select the most appropriate option:
- Very Inaccurate: The statement is definitely false or the participant would strongly disagree with it.
- Moderately Inaccurate: The statement is mostly false or the participant would generally disagree with it.
- Neither Accurate nor Inaccurate: The participant would be neutral on the statement, cannot decide, or find the statement equally true and false.
- Moderately Accurate: The statement is mostly true or the participant would generally agree with it.
- Very Accurate: The statement is definitely true or the participant would strongly agree with it.

IPIP-NEO-120 question to answer:
{question}

Participant's daily diaries:
{thoughts}

Your response must be exactly one of:
Very Inaccurate
Moderately Inaccurate
Neither Accurate nor Inaccurate
Moderately Accurate
Very Accurate

Do not include any explanation, punctuation, or additional text. Return only the exact phrase from the list above.
"""

# create a single instance of ChatGPT 
# so that we can keep track of running costs
chat = ChatGPT(model_provider_order=MODEL_TO_EVALUATE)

response, message_history_id = example_messaging_wrapper(chat, system_role, message)

INFO:root:processing message with 307 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Messages to GPT:
[{'role': 'system', 'content': 'you are a helpful assistant.'}, {'role': 'user', 'content': "\nSimulate your response to the prompt below without being provided the question and thoughts. This is for testing purposes.\n\n---\n\nYour task is to respond to the following IPIP-NEO-120 question based on the participant's daily diaries of the most significant event that occurred during the day, provided below. Respond as though you are the individual who generated these thoughts, reflecting their personality traits.\nBase your answer on inferred personality traits. Think carefully about what the thoughts imply about tendencies and behaviors.\n\nFor each question, select the most appropriate option:\n- Very Inaccurate: The statement is definitely false or the participant would strongly disagree with it.\n- Moderately Inaccurate: The statement is mostly false or the participant would generally disagree with it.\n- Neither Accurate nor Inaccurate: The participant would be neutr

# AAPECS

In [12]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import time
from scipy import stats

pd.options.display.max_columns = None

In [13]:
questions_list = [
    {'text': 'Worry about things', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Make friends easily', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Have a vivid imagination', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Trust others', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Complete tasks successfully', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Get angry easily', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Love large parties', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Believe in the importance of art', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Use others for my own ends', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Like to tidy up', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Often feel blue', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Take charge', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Experience my emotions intensely', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Love to help others', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Keep my promises', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Find it difficult to approach others', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Am always busy', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Prefer variety to routine', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Love a good fight', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Work hard', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Go on binges', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Love excitement', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Love to read challenging material', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Believe that I am better than others', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Am always prepared', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Panic easily', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Radiate joy', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Tend to vote for liberal political candidates', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Sympathize with the homeless', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Jump into things without thinking', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Fear for the worst', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Feel comfortable around people', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Enjoy wild flights of fantasy', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Believe that others have good intentions', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Excel in what I do', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Get irritated easily', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Talk to a lot of different people at parties', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'See beauty in things that others might not notice', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Cheat to get ahead', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Often forget to put things back in their proper place', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Dislike myself', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Try to lead others', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Feel others\' emotions', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Am concerned about others', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Tell the truth', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Am afraid to draw attention to myself', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Am always on the go', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Prefer to stick with things that I know', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Yell at people', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Do more than what\'s expected of me', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Rarely overindulge', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Seek adventure', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Avoid philosophical discussions', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Think highly of myself', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Carry out my plans', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Become overwhelmed by events', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Have a lot of fun', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Believe that there is no absolute right and wrong', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Feel sympathy for those who are worse off than myself', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Make rash decisions', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Am afraid of many things', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Avoid contacts with others', 'keyed': 'minus', 'domain': 'E'},
    {'text': 'Love to daydream', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Trust what people say', 'keyed': 'plus', 'domain': 'A'},
    {'text': 'Handle tasks smoothly', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Lose my temper', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Prefer to be alone', 'keyed': 'minus', 'domain': 'E'},
    {'text': 'Do not like poetry', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Take advantage of others', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Leave a mess in my room', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Am often down in the dumps', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Take control of things', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Rarely notice my emotional reactions', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Am indifferent to the feelings of others', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Break rules', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Only feel comfortable with friends', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Do a lot in my spare time', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Dislike changes', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Insult people', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Do just enough work to get by', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Easily resist temptations', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Enjoy being reckless', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Have difficulty understanding abstract ideas', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Have a high opinion of myself', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Waste my time', 'keyed': 'minus', 'domain': 'C'},
    {'text': "Feel that I'm unable to deal with things", 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Love life', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Tend to vote for conservative political candidates', 'keyed': 'minus', 'domain': 'O'},
    {'text': "Am not interested in other people's problems", 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Rush into things', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Get stressed out easily', 'keyed': 'plus', 'domain': 'N'},
    {'text': 'Keep others at a distance', 'keyed': 'minus', 'domain': 'E'},
    {'text': 'Like to get lost in thought', 'keyed': 'plus', 'domain': 'O'},
    {'text': 'Distrust people', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Know how to get things done', 'keyed': 'plus', 'domain': 'C'},
    {'text': 'Am not easily annoyed', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Avoid crowds', 'keyed': 'minus', 'domain': 'E'},
    {'text': 'Do not enjoy going to art museums', 'keyed': 'minus', 'domain': 'O'},
    {'text': "Obstruct others' plans", 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Leave my belongings around', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Feel comfortable with myself', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Wait for others to lead the way', 'keyed': 'minus', 'domain': 'E'},
    {'text': "Don't understand people who get emotional", 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Take no time for others', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Break my promises', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Am not bothered by difficult social situations', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Like to take it easy', 'keyed': 'minus', 'domain': 'E'},
    {'text': 'Am attached to conventional ways', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Get back at others', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Put little time and effort into my work', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Am able to control my cravings', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Act wild and crazy', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Am not interested in theoretical discussions', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Boast about my virtues', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Have difficulty starting tasks', 'keyed': 'minus', 'domain': 'C'},
    {'text': 'Remain calm under pressure', 'keyed': 'minus', 'domain': 'N'},
    {'text': 'Look at the bright side of life', 'keyed': 'plus', 'domain': 'E'},
    {'text': 'Believe that we should be tough on crime', 'keyed': 'minus', 'domain': 'O'},
    {'text': 'Try not to think about the needy', 'keyed': 'minus', 'domain': 'A'},
    {'text': 'Act without thinking', 'keyed': 'minus', 'domain': 'C'}
]

for i in range(len(questions_list)):
    questions_list[i]['text'] = f'Question {i+1}: ' + questions_list[i]['text']

In [16]:
pheno_df = pd.read_csv(f'{DATA_ROOT}/eod_new_time.csv')
data_root = f'{DATA_ROOT}/raw_video_logs'
data_files = [f for f in listdir(data_root) if isfile(join(data_root, f))]

sub_transcripts = {}
sub_lengths = {}

for file in tqdm(data_files):
    df = pd.read_csv(f'{data_root}/{file}')
    df = df[(df.values[:, -1] != 'NO_ANSWER') & (df.values[:, -1] != 'SKIPPED')]

    dates = [x.replace('/', '_') for x in df['Survey Submitted Date'].values]  # dd_mm_yyyy
    times = [x.replace(':', '-') for x in df['Survey Submitted Time'].values]  # dd_mm_yyyy
    addresses = df.values[:, -1]
    userid = df['User Id'].values
    usernum = file.lower().replace('eod', '').replace('vids', '').replace('videos', '').replace('.csv', '').replace('video', '')
    usernum = int(usernum)
    triggers = [x.replace(' ', '') for x in df['Trigger Type'].values]

    assert np.all(['http' in x for x in addresses])
    assert np.all((df['Trigger Type'].values == "DAILY") | (df['Trigger Type'].values == "DELETED TRIGGER") | (df['Trigger Type'].values == "ONCE"))
    assert np.all([x.split('/')[-1].split('.')][-1] == 'mp4' for x in addresses)
    
    sub_pheno_df = pheno_df[pheno_df.participantID == usernum]
    if sub_pheno_df.shape[0] == 0: continue

    
    for i in range(len(addresses)):
        txt = json.load(open(f'{DATA_ROOT}/transcripts_json/{usernum}/{usernum}_{dates[i]}_{times[i]}_{triggers[i]}_{userid[i]}.json'))['text'].strip()
        
        day, month, year = dates[i].split('_')
        sub = usernum
        if sub not in sub_transcripts:
            sub_transcripts[sub] = []
        sub_transcripts[sub].append(txt)
        
        recording_length = json.load(open(f'{DATA_ROOT}/transcripts_json/{usernum}/{usernum}_{dates[i]}_{times[i]}_{triggers[i]}_{userid[i]}.json'))['segments'][-1]['end']
        if sub not in sub_lengths:
            sub_lengths[sub] = []
        sub_lengths[sub].append(recording_length)

A Jupyter Widget

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [18]:
def get_NEO_FFI_prompt(thoughts, question):
    prompt_template = f"""
Your task is to respond to the following IPIP-NEO-120 question based on the participant's daily diaries of the most significant event that occurred during the day, provided below. Respond as though you are the individual who generated these thoughts, reflecting their personality traits.
Base your answer on inferred personality traits. Think carefully about what the thoughts imply about tendencies and behaviors.

For each question, select the most appropriate option:
- Very Inaccurate: The statement is definitely false or the participant would strongly disagree with it.
- Moderately Inaccurate: The statement is mostly false or the participant would generally disagree with it.
- Neither Accurate nor Inaccurate: The participant would be neutral on the statement, cannot decide, or find the statement equally true and false.
- Moderately Accurate: The statement is mostly true or the participant would generally agree with it.
- Very Accurate: The statement is definitely true or the participant would strongly agree with it.

IPIP-NEO-120 question to answer:
{question}

Participant's daily diaries:
{thoughts}

Your response must be exactly one of:
Very Inaccurate
Moderately Inaccurate
Neither Accurate nor Inaccurate
Moderately Accurate
Very Accurate

Do not include any explanation, punctuation, or additional text. Return only the exact phrase from the list above.
"""
    return prompt_template

In [19]:
def generic_messaging_wrapper(chat, system_role, message):
    # with halt_on_error set to True in ChatGPT class, 
    # we use exception propogation to handle errors and edge-cases
    message_history_id = -1
    required_values = None
    try:
        response, message_history_id = chat.send_message(system_role=system_role,
                                                         message=message)
        assert response is not None
        response_json = json.loads(response.choices[0].message.content)
        required_values = response_json['response']
    except Exception as e:
        response_json = {}
        chat.logger.info(f'Messaging wrapper failure - {str(e)}')
        print(traceback.format_exc())

    cost, n_prompt_tokens, n_completion_tokens = chat.get_running_cost_num_prompt_completion_tokens()
    return required_values, cost

In [20]:
system_role = ''
print(system_role)

In [21]:
chat = ChatGPT(model_provider_order=MODEL_TO_EVALUATE)

thoughts = '\n'.join(sub_transcripts[54])
domain_questions = [x['text'] for x in questions_list if x['domain']=='N']
message = get_NEO_FFI_prompt(thoughts, domain_questions[0])
required_values, cost = generic_messaging_wrapper(chat, system_role, message)

print(required_values)

INFO:root:processing message with 3036 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Very Accurate


In [24]:
from multiprocessing import Process, Manager

def multiproc_neo_wrapper(thoughts, question_i, name, return_dict, model_to_evaluate):
    try:
        chat = ChatGPT(model_provider_order=model_to_evaluate)
        dim_question = questions_list[question_i]['text']
        message = get_NEO_FFI_prompt(thoughts, dim_question)
        required_values, cost = generic_messaging_wrapper(chat, system_role, message)
        return_dict[f'{name}'] = (required_values, cost)
    except Exception as e:
        pass  # silent failures 

In [25]:
return_dict = Manager().dict()

In [26]:
import resource

soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print("Soft limit:", soft, "Hard limit:", hard)
# Raise soft limit (if you have permission):
resource.setrlimit(resource.RLIMIT_NOFILE, (30000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print("New Soft limit:", soft, "New Hard limit:", hard)

Soft limit: 4096 Hard limit: 262144
New Soft limit: 30000 New Hard limit: 262144


In [30]:
n_subs = len(sub_transcripts)
total_requests = np.sum([min(14, len(sub_transcripts[x]))/2 for x in sub_transcripts]) * len(questions_list)
total_requests

75420.0

In [33]:
def run_until_resolved(good_th_dict, questions, return_dict, model_to_evaluate,
                       max_attempts=5, sleep_between=0.1, max_concurrent_calls=10000):
    n_subs = len(good_th_dict)
    total_requests = np.sum([min(14, len(good_th_dict[x])) for x in good_th_dict]) * len(questions)
    start_time = time.time()
    
    def format_time(seconds):
        mins, secs = divmod(int(seconds), 60)
        hrs, mins = divmod(mins, 60)
        return f"{hrs:02d}:{mins:02d}:{secs:02d}"
    
    for attempt in range(1, max_attempts + 1):
        procs = []
        n_missing = 0
        missing_items = []

        # Check for missing entries
        for subject in good_th_dict:
            for question_i in range(len(questions)):
                for t_i in range(1, min(15, len(good_th_dict[subject]) + 1), 2):
                    name = f'{subject}-{question_i}-{t_i}'
                    if name not in return_dict:
                        n_missing += 1
                        missing_items.append((subject, question_i, t_i, name))
                    else:
                        if return_dict[name][0] is None:
                            n_missing += 1
                            missing_items.append((subject, question_i, t_i, name))

        if n_missing == 0:
            print("All responses successfully completed.")
            break
        else:
            print(f"Attempt {attempt}: {n_missing} / {total_requests} requests missing.")

        # Dispatch missing jobs
        for (subject_i, (subject, question_i, t_i, name)) in enumerate(missing_items):
            transcript = '\n'.join([x.replace('\n', '') for x in good_th_dict[subject]][:t_i])
            proc = Process(target=multiproc_neo_wrapper, args=(transcript, question_i, name, return_dict, model_to_evaluate))
            proc.start()
            procs.append(proc)
            time.sleep(sleep_between)

            if len(procs) >= max_concurrent_calls:
                for proc in procs:
                    proc.join()
                procs = []
            
                # Intermediate update
                completed_items = dict(return_dict)
                running_cost = np.sum([completed_items[x][-1] for x in completed_items])
                n_completed = len(completed_items)
                avg_cost = running_cost / n_completed if n_completed else 0
                estimated_total_cost = avg_cost * total_requests

                elapsed_time = time.time() - start_time
                avg_time_per_call = elapsed_time / n_completed if n_completed else 0
                remaining_calls = total_requests - n_completed
                estimated_time_remaining = avg_time_per_call * remaining_calls

                clear_output(wait=True)
                print(f"Attempt {attempt}")
                print(f"Completed: {n_completed}/{total_requests}")
                print(f"Running cost: ${running_cost:.3f}")
                print(f"Estimated total cost: ${estimated_total_cost:.3f}")
                print(f"Elapsed time: {format_time(elapsed_time)}")
                print(f"Estimated time remaining: {format_time(estimated_time_remaining)}")
                
        # Final join
        for proc in procs:
            proc.join()

        running_cost = np.sum([return_dict[x][-1] for x in return_dict])
        clear_output(wait=True)
        print(f"Total running cost: {running_cost:.3f}")
        

        clear_output(wait=True)
        n_missing = 0
        # Check for missing entries
        for subject in good_th_dict:
            for question_i in range(len(questions)):
                for t_i in range(1, min(15, len(good_th_dict[subject]) + 1), 2):
                    name = f'{subject}-{question_i}-{t_i}'
                    if name not in return_dict:
                        n_missing += 1
        completed_dict = dict(return_dict)
        completed_items = [v for v in completed_dict.values()]
        total_cost = np.sum([x[-1] for x in completed_items])
        n_completed = len(completed_items)
        avg_cost = total_cost / n_completed if n_completed else 0
        total_elapsed_time = time.time() - start_time

        print(f"Total responses expected: {total_requests}")
        print(f"Successful: {n_completed}")
        print(f"Failed: {n_missing}")
        print(f"Total cost: ${total_cost:.3f}")
        print(f"Avg cost per response: ${avg_cost:.4f}")
        print(f"Total runtime: {format_time(total_elapsed_time)}")
        
    else:
        n_missing = 0
        # Check for missing entries
        for subject in good_th_dict:
            for question_i in range(len(questions)):
                for t_i in range(1, min(15, len(good_th_dict[subject]) + 1), 2):
                    name = f'{subject}-{question_i}-{t_i}'
                    if name not in return_dict:
                        n_missing += 1
                    else:
                        if return_dict[name][0] is None:
                            n_missing += 1
                            missing_items.append((subject, question_i, name))

        print(f'!!! Max attempts reached. {n_missing} requests are still unresolved. !!!')

In [34]:
MODEL_TO_EVALUATE

('openai/gpt-4.1-mini', ['OpenAI'], 'gpt_41_mini')

In [ ]:
run_until_resolved(sub_transcripts, questions_list, return_dict, MODEL_TO_EVALUATE)

Attempt 1
Completed: 20000/150840
Running cost: $8.518
Estimated total cost: $64.243
Elapsed time: 00:36:42
Estimated time remaining: 04:00:08


INFO:root:processing message with 1327 tokens...
INFO:root:processing message with 1641 tokens...
INFO:root:processing message with 2184 tokens...
INFO:root:processing message with 2633 tokens...
INFO:root:processing message with 3001 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:processing message with 612 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:processing message with 935 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:processing message with 1327 tokens...
INFO:root:processing message with 1641 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:processing message with 2184 tokens...
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:processing message with 2633 tok

In [ ]:
out = dict(return_dict)
pickle.dump(out, open(f'{DATA_ROOT}/aapecs_return_dict_windowed.pickle', 
                      'wb'))

In [37]:
n_subs = len(sub_transcripts)
n_missing = 0

for (subject_i, subject) in enumerate(sub_transcripts):
    for question_i in range(len(questions_list)):
        for t_i in range(1, min(15, len(sub_transcripts[subject]) + 1), 2):
            name = f'{subject}-{question_i}-{t_i}'
            if name not in return_dict:
                n_missing += 1
                print(name)
            
assert n_missing == 0

In [52]:
def map_responses_to_integers(response_list, reverse_coding_list):
    # Define the mapping from response options to integers
    response_mapping = {
        'Very Inaccurate': 0,
        'Moderately Inaccurate': 1,
        'Neither Accurate nor Inaccurate': 2,
        'Moderately Accurate': 3,
        'Very Accurate': 4,
        None: np.nan,
        np.nan: np.nan
    }
    
    reverse_mapping = {
        'Very Inaccurate': 4,
        'Moderately Inaccurate': 3,
        'Neither Accurate nor Inaccurate': 2,
        'Moderately Accurate': 1,
        'Very Accurate': 0,
        None: np.nan,
        np.nan: np.nan
    }

    # Iterate through both lists and apply the appropriate mapping based on reverse coding
    mapped_responses = [
        reverse_mapping[response] if reverse_coding else response_mapping[response]
        for response, reverse_coding in zip(response_list, reverse_coding_list)
    ]

    return mapped_responses

In [53]:
out_dict = dict()
sub_ids = []
for subject in sub_transcripts:
    for t_i in range(1, min(15, len(sub_transcripts[subject]) + 1), 2):
        for question_i in range(1, len(questions_list)+1):
            if subject not in out_dict:
                out_dict[subject] = {}
            v = return_dict[f'{subject}-{question_i-1}-{t_i}'][0]
            if v is None: raise Exception(f'{subject}-{question_i} is None')
            out_dict[subject][f'question_{question_i}_{t_i}']  = v
    sub_ids.append(f'{subject}')

In [54]:
out_df = pd.DataFrame(out_dict).T
out_df['subject'] = pd.DataFrame(out_dict).T.index
out_df['participantID'] = sub_ids
out_df.head()

,question_1_1,question_2_1,question_3_1,question_4_1,question_5_1,question_6_1,question_7_1,question_8_1,question_9_1,question_10_1,question_11_1,question_12_1,question_13_1,question_14_1,question_15_1,question_16_1,question_17_1,question_18_1,question_19_1,question_20_1,question_21_1,question_22_1,question_23_1,question_24_1,question_25_1,question_26_1,question_27_1,question_28_1,question_29_1,question_30_1,question_31_1,question_32_1,question_33_1,question_34_1,question_35_1,question_36_1,question_37_1,question_38_1,question_39_1,question_40_1,question_41_1,question_42_1,question_43_1,question_44_1,question_45_1,question_46_1,question_47_1,question_48_1,question_49_1,question_50_1,question_51_1,question_52_1,question_53_1,question_54_1,question_55_1,question_56_1,question_57_1,question_58_1,question_59_1,question_60_1,question_61_1,question_62_1,question_63_1,question_64_1,question_65_1,question_66_1,question_67_1,question_68_1,question_69_1,question_70_1,question_71_1,question_72_1,question_73_1,question_74_1,question_75_1,question_76_1,question_77_1,question_78_1,question_79_1,question_80_1,question_81_1,question_82_1,question_83_1,question_84_1,question_85_1,question_86_1,question_87_1,question_88_1,question_89_1,question_90_1,question_91_1,question_92_1,question_93_1,question_94_1,question_95_1,question_96_1,question_97_1,question_98_1,question_99_1,question_100_1,question_101_1,question_102_1,question_103_1,question_104_1,question_105_1,question_106_1,question_107_1,question_108_1,question_109_1,question_110_1,question_111_1,question_112_1,question_113_1,question_114_1,question_115_1,question_116_1,question_117_1,question_118_1,question_119_1,question_120_1,question_1_3,question_2_3,question_3_3,question_4_3,question_5_3,question_6_3,question_7_3,question_8_3,question_9_3,question_10_3,question_11_3,question_12_3,question_13_3,question_14_3,question_15_3,question_16_3,question_17_3,question_18_3,question_19_3,question_20_3,question_21_3,question_22_3,question_23_3,question_24_3,question_25_3,question_26_3,question_27_3,question_28_3,question_29_3,question_30_3,question_31_3,question_32_3,question_33_3,question_34_3,question_35_3,question_36_3,question_37_3,question_38_3,question_39_3,question_40_3,question_41_3,question_42_3,question_43_3,question_44_3,question_45_3,question_46_3,question_47_3,question_48_3,question_49_3,question_50_3,question_51_3,question_52_3,question_53_3,question_54_3,question_55_3,question_56_3,question_57_3,question_58_3,question_59_3,question_60_3,question_61_3,question_62_3,question_63_3,question_64_3,question_65_3,question_66_3,question_67_3,question_68_3,question_69_3,question_70_3,question_71_3,question_72_3,question_73_3,question_74_3,question_75_3,question_76_3,question_77_3,question_78_3,question_79_3,question_80_3,question_81_3,question_82_3,question_83_3,question_84_3,question_85_3,question_86_3,question_87_3,question_88_3,question_89_3,question_90_3,question_91_3,question_92_3,question_93_3,question_94_3,question_95_3,question_96_3,question_97_3,question_98_3,question_99_3,question_100_3,question_101_3,question_102_3,question_103_3,question_104_3,question_105_3,question_106_3,question_107_3,question_108_3,question_109_3,question_110_3,question_111_3,question_112_3,question_113_3,question_114_3,question_115_3,question_116_3,question_117_3,question_118_3,question_119_3,question_120_3,question_1_5,question_2_5,question_3_5,question_4_5,question_5_5,question_6_5,question_7_5,question_8_5,question_9_5,question_10_5,question_11_5,question_12_5,question_13_5,question_14_5,question_15_5,question_16_5,question_17_5,question_18_5,question_19_5,question_20_5,question_21_5,question_22_5,question_23_5,question_24_5,question_25_5,question_26_5,question_27_5,question_28_5,question_29_5,question_30_5,question_31_5,question_32_5,question_33_5,question_34_5,question_35_5,question_36_5,question_37_5,question_38_5,question_39_5,question_40_5,question_41_5,question_42_5,question_43_5,question_44_5,question

In [55]:
name_mapping_aapecs = {
    'O': 'neoOpenness',
    'C': 'neoConscientiousness',
    'E': 'neoExtraversion',
    'A': 'neoAgreeableness',
    'N': 'neoNeuroticism'
}

gpt_neo_scores = pd.DataFrame()
gpt_neo_scores['participantID'] = out_df.index.values
for dim in ['O', 'C', 'E', 'A', 'N']:
    dim_questions = [x['text'].split(':')[0].lower().replace(' ', '_') for x in questions_list if x['domain']==dim]
    is_reverse_coded = [questions_list[q_i-1]['keyed']!='plus' for q_i in [int(x.split('_')[1]) for x in dim_questions]]
    for t_i in range(1, 15, 2):
        dim_questions_t_i = [x+f'_{t_i}' for x in dim_questions]
        v = [np.mean(map_responses_to_integers(a, is_reverse_coded)) for a in out_df[dim_questions_t_i].values]
        gpt_neo_scores[name_mapping_aapecs[dim]+f'_{t_i}'] = v

In [56]:
gpt_neo_scores.head()

,participantID,neoOpenness_1,neoOpenness_3,neoOpenness_5,neoOpenness_7,neoOpenness_9,neoOpenness_11,neoOpenness_13,neoConscientiousness_1,neoConscientiousness_3,neoConscientiousness_5,neoConscientiousness_7,neoConscientiousness_9,neoConscientiousness_11,neoConscientiousness_13,neoExtraversion_1,neoExtraversion_3,neoExtraversion_5,neoExtraversion_7,neoExtraversion_9,neoExtraversion_11,neoExtraversion_13,neoAgreeableness_1,neoAgreeableness_3,neoAgreeableness_5,neoAgreeableness_7,neoAgreeableness_9,neoAgreeableness_11,neoAgreeableness_13,neoNeuroticism_1,neoNeuroticism_3,neoNeuroticism_5,neoNeuroticism_7,neoNeuroticism_9,neoNeuroticism_11,neoNeuroticism_13
0,85,1.916667,1.625000,NaN,NaN,NaN,NaN,NaN,2.958333,2.916667,NaN,NaN,NaN,NaN,NaN,2.125000,1.500000,NaN,NaN,NaN,NaN,NaN,2.750000,2.625000,NaN,NaN,NaN,NaN,NaN,1.875000,2.375000,NaN,NaN,NaN,NaN,NaN
1,54,1.791667,2.166667,2.000000,2.000000,1.916667,2.166667,2.208333,2.958333,3.166667,3.125000,3.125000,3.125000,3.208333,3.083333,1.625000,2.375000,2.166667,2.250000,2.083333,2.083333,2.083333,2.750000,2.750000,2.750000,2.708333,2.791667,2.750000,2.750000,2.166667,1.833333,2.041667,1.875000,2.000000,2.125000,2.291667
2,78,2.375000,2.416667,2.291667,2.083333,2.291667,2.500000,2.250000,2.791667,2.708333,3.000000,3.000000,3.041667,3.000000,2.958333,1.500000,1.291667,2.166667,2.166667,2.125000,2.166667,1.708333,2.750000,2.750000,2.791667,2.958333,2.916667,3.000000,2.958333,2.666667,2.875000,2.333333,2.083333,2.083333,2.541667,3.000000
3,97,1.791667,2.833333,2.541667,2.458333,2.583333,2.708333,2.625000,2.958333,3.041667,3.083333,3.208333,3.125000,3.041667,3.041667,1.416667,2.250000,2.250000,2.208333,2.083333,2.166667,2.416667,2.791667,2.750000,2.958333,3.041667,3.041667,3.041667,3.041667,1.916667,1.333333,1.416667,1.333333,1.500000,1.750000,1.416667
4,48,2.166667,2.000000,2.208333,2.000000,1.666667,2.083333,2.125000,3.291667,3.458333,3.250000,3.250000,3.166667,3.250000,3.250000,2.125000,2.416667,2.375000,2.250000,2.250000,2.083333,2.291667,2.791667,2.833333,3.041667,2.916667,2.875000,2.916667,2.833333,0.833333,0.916667,0.958333,1.083333,1.083333,1.250000,1.250000


In [57]:
model_canonical_name = MODEL_TO_EVALUATE[-1]
gpt_neo_scores.to_csv(f'{DATA_ROOT}/aapecs_{model_canonical_name}_text_per_question_scores_windowed.csv')